# Code Validation - Model Experiments

## Contents:<a class="anchor" id="contents"></a>
* [Setup and Data Prep](#setup)
* [Equivariance Tests](#equivariance)
* [DenseNet Depth Tests](#depth)
* [DenseNet Traianble Parameters](#params)

## Setup and Data Prep <a class="anchor" id="setup"></a>
----------------------------------

In [1]:
import time
import tensorflow as tf
import keras.backend as K
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from keras import models, layers, optimizers, regularizers
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.utils import to_categorical
from keras import Input
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import keras
import os, shutil
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras_gcnn.layers import GConv2D, GBatchNorm
from keras_gcnn.layers.pooling import GroupPool

from keras.utils import np_utils
from keras.layers import concatenate
from keras.layers import Activation
from keras.layers import Cropping2D # Need this for concatenate to work with valid pooling
from keras.regularizers import l2 # Weight decay from the DenseNet paper

from ipynb.fs.full.my_functions import build_and_compile_model, build_and_compile_model_GCNN, fit_model_to_generator, plot_auc, area_under_ROC_curve
from ipynb.fs.full.my_functions import plot_results,plot_graphs, plot_smooth, plot_smooth_graphs, fit_model_to_directory_generator, evaluate_auc
from ipynb.fs.full.my_functions import build_and_compile_dense_model, crop, memory_required, count_conv_layers
from ipynb.fs.full.my_functions import model_ensemble_evaluation, acc_comparison, save_history, load_history, equi_check

Using TensorFlow backend.


In [2]:
base_dir = "C:/GitRepos/FINAL PROJECT DATA/Histopathologic Cancer Detection/WholePCamSetFromGithub/converted_images/"
tr_dir = os.path.join(base_dir, "train")
va_dir = os.path.join(base_dir, "valid")
te_dir = os.path.join(base_dir, "test")

In [3]:
batch_size = 64 
targ_size = (96,96)
classification = "binary"

train_datagen = ImageDataGenerator(rescale=1./255)

test_generator = train_datagen.flow_from_directory(
    te_dir,
    target_size=targ_size,
    batch_size=batch_size,
    class_mode=classification)


Found 32768 images belonging to 2 classes.


## Equivariance Tests <a class="anchor" id="setup"></a>
----------------------------------

In [4]:
A = build_and_compile_model(12,["maxpool","conv12","maxpool","conv12","maxpool","globalavg"])


A_GCNN_P4 = build_and_compile_model_GCNN("C4",6,["maxpool","conv6","maxpool","conv6","maxpool","grouppool","globalavg"])


A_GCNN_P4M = build_and_compile_model_GCNN("D4",4,["maxpool","conv4","maxpool","conv4","maxpool","grouppool","globalavg"])


B_Dense = build_and_compile_dense_model(24,growth_rate=12,dense_blocks=5,conv_layers=1,
                                           is_gconv=False,padding="same",dropout=0.2,img_size=96,
                                           opt="Adam",weight_decay=1e-4,output="sigmoid",labels=1,bc_model=False)


B_Dense_GCNN_P4 = build_and_compile_dense_model(12,growth_rate=6,dense_blocks=5,conv_layers=1,
                                           is_gconv=True,gconv_type="C4",padding="valid",dropout=0.2,img_size=96,
                                           opt="Adam",weight_decay=1e-4,output="sigmoid",labels=1,bc_model=False)

In [5]:
A.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 12)        336       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 12)        1308      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 12)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 12)        1308      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 12)        0         
__________

In [6]:
A_GCNN_P4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
g_conv2d_1 (GConv2D)         (None, 96, 96, 24)        162       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 24)        0         
_________________________________________________________________
g_conv2d_2 (GConv2D)         (None, 48, 48, 24)        1296      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 24)        0         
_________________________________________________________________
g_conv2d_3 (GConv2D)         (None, 24, 24, 24)        1296      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 24)        0         
__________

In [7]:
A_GCNN_P4M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
g_conv2d_4 (GConv2D)         (None, 96, 96, 32)        108       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
g_conv2d_5 (GConv2D)         (None, 48, 48, 32)        1152      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
g_conv2d_6 (GConv2D)         (None, 24, 24, 32)        1152      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 32)        0         
__________

In [8]:
B_Dense.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 96, 96, 24)   672         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 24)   96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 96, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_5 (

In [9]:
B_Dense_GCNN_P4.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
g_conv2d_7 (GConv2D)            (None, 94, 94, 48)   324         input_5[0][0]                    
__________________________________________________________________________________________________
g_batch_norm_1 (GBatchNorm)     (None, 94, 94, 48)   48          g_conv2d_7[0][0]                 
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 94, 94, 48)   0           g_batch_norm_1[0][0]             
__________________________________________________________________________________________________
g_conv2d_8

In [12]:
batch = test_generator.next()

In [19]:
print("Model A:",equi_check(A,batch[0]))
print("Model A_GCNN_P4:",equi_check(A_GCNN_P4,batch[0]))
print("Model A_GCNN_P4M:",equi_check(A_GCNN_P4M,batch[0]))
print("Model B_Dense:",equi_check(B_Dense,batch[0]))
print("Model B_Dense_GCNN_P4:",equi_check(B_Dense_GCNN_P4,batch[0]))

Model A: False
Model A_GCNN_P4: True
Model A_GCNN_P4M: True
Model B_Dense: False
Model B_Dense_GCNN_P4: True


## DenseNet Depth Testing <a class="anchor" id="setup"></a>
----------------------------------

In [5]:
def depth_check(model):
    depth = 0
    for layer in model.layers:
        if "conv2d" in layer.get_config()["name"].lower():
            depth +=1
    return depth+1 # +1 for the final activation

In [8]:
a1,b1,b2,b3 = 3,12,32,41

In [9]:
DenseNet40 = build_and_compile_dense_model(16,growth_rate=12,dense_blocks=a1,conv_layers=b1,
                                           is_gconv=False,padding="same",dropout=0.2,img_size=96,
                                           opt="Adam",weight_decay=1e-4,output="sigmoid",labels=1,bc_model=False)

In [54]:
DenseNet100_GCNN = build_and_compile_dense_model(8,growth_rate=6,dense_blocks=a1,conv_layers=b2,
                                           is_gconv=True,gconv_type="C4",padding="same",dropout=0.2,img_size=96,
                                           opt="Adam",weight_decay=1e-4,output="sigmoid",labels=1,bc_model=False)

In [55]:
DenseNet_BC250 = build_and_compile_dense_model(16,growth_rate=24,dense_blocks=a1,conv_layers=b3,
                                           is_gconv=False,padding="same",dropout=0.2,img_size=96,
                                           opt="Adam",weight_decay=1e-4,output="sigmoid",labels=1,bc_model=True)

In [10]:
DenseNet40.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 96, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
DenseNet100_GCNN.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
g_conv2d_1 (GConv2D)            (None, 96, 96, 32)   216         input_2[0][0]                    
__________________________________________________________________________________________________
g_batch_norm_1 (GBatchNorm)     (None, 96, 96, 32)   32          g_conv2d_1[0][0]                 
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 96, 96, 32)   0           g_batch_norm_1[0][0]             
__________________________________________________________________________________________________
g_conv2d_2

In [15]:
DenseNet_BC250.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_289 (Conv2D)             (None, 96, 96, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_289 (BatchN (None, 96, 96, 16)   64          conv2d_289[0][0]                 
__________________________________________________________________________________________________
activation_382 (Activation)     (None, 96, 96, 16)   0           batch_normalization_289[0][0]    
__________________________________________________________________________________________________
conv2d_290

__________________________________________________________________________________________________
concatenate_357 (Concatenate)   (None, 24, 24, 1472) 0           concatenate_356[0][0]            
                                                                 dropout_357[0][0]                
__________________________________________________________________________________________________
batch_normalization_495 (BatchN (None, 24, 24, 1472) 5888        concatenate_357[0][0]            
__________________________________________________________________________________________________
activation_586 (Activation)     (None, 24, 24, 1472) 0           batch_normalization_495[0][0]    
__________________________________________________________________________________________________
conv2d_496 (Conv2D)             (None, 24, 24, 96)   141408      activation_586[0][0]             
__________________________________________________________________________________________________
batch_norm

In [58]:
a1,b1,b2,b3 = 3,12,32,41

assert depth_check(DenseNet40) == a1*b1+a1+1
assert depth_check(DenseNet100_GCNN) == a1*b2+a1+1
assert depth_check(DenseNet_BC250) == a1*b3*2+a1+1

print("DenseNet40 depth: ",depth_check(DenseNet40))
print("DenseNet100_GCNN depth: ",depth_check(DenseNet100_GCNN))
print("DenseNet_BC250 depth: ",depth_check(DenseNet_BC250))

DenseNet40 depth:  40
DenseNet100_GCNN depth:  100
DenseNet_BC250 depth:  250


## Trainable Parameters <a class="anchor" id="params"></a>
----------------------------------

In [8]:
# Extracted from the source code of Keras model.summary() and turned into a standalone function.
def get_trainable_weights(model):
    return np.sum([K.count_params(p) for p in set(model.trainable_weights)])

In [ ]:
# Adding a model from the rotational equivariance paper for testing.

P4_DenseNet = build_and_compile_dense_model(16,growth_rate=12,dense_blocks=5,conv_layers=1,
                                           is_gconv=True,gconv_type="C4",padding="same",dropout=0.2,img_size=96,
                                           opt="Adam",weight_decay=1e-4,output="sigmoid",labels=1,bc_model=False)

In [51]:
P4_DenseNet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
g_conv2d_150 (GConv2D)          (None, 96, 96, 64)   432         input_10[0][0]                   
__________________________________________________________________________________________________
g_batch_norm_150 (GBatchNorm)   (None, 96, 96, 64)   64          g_conv2d_150[0][0]               
__________________________________________________________________________________________________
activation_659 (Activation)     (None, 96, 96, 64)   0           g_batch_norm_150[0][0]           
__________________________________________________________________________________________________
g_conv2d_1

In [60]:
# Ensuring the trainable weights are within 10 percent of those listed in the DenseNet paper.
# Slgiht variance are due to G-CNN not being directly comparable, and the compression levels varying in the BC model.

assert abs(get_trainable_weights(DenseNet40)-1000000) < 1000000*0.1
assert abs(get_trainable_weights(DenseNet100_GCNN)-7000000) < 7000000*0.1
assert abs(get_trainable_weights(DenseNet_BC250)-15300000) < 15300000*0.1
assert abs(get_trainable_weights(P4_DenseNet)-125000) < 125000*0.1

print("DenseNet40 weights:",get_trainable_weights(DenseNet40), "expected:",1000000)
print("DenseNet100_GCNN weights:",get_trainable_weights(DenseNet100_GCNN), "expected:",7000000)
print("DenseNet_BC250 weights:",get_trainable_weights(DenseNet_BC250), "expected:",15300000)
print("P4_DenseNet weights:", get_trainable_weights(P4_DenseNet), "expected:",125000)

DenseNet40 weights: 1016593 expected: 1000000
DenseNet100_GCNN weights: 6909713 expected: 7000000
DenseNet_BC250 weights: 16418933 expected: 15300000
P4_DenseNet weights: 124565 expected: 125000
